In [42]:
# generate dummy data for training
import numpy as np
from keras.utils.np_utils import to_categorical

data_rows = 1000
X_train = np.random.random((data_rows, 4096))
y_train = np.random.randint(2, size=(data_rows, 1))
y_train = to_categorical(y_train, 2)

print(X_train.shape)
print(y_train.shape)


(1000, 4096)
(1000, 2)


## Loading all data and labels from file

In [107]:
# Loading all data and labels from file
import h5py
import numpy as np
from keras.utils.np_utils import to_categorical

feature_file = "../preprocess/MSRII-c3d-features.h5"
mode = "r"
with h5py.File(feature_file, mode) as h5:
        '''
        print(h5.keys())
        print(h5['features'])
        print(h5['labels'])
        '''
        data = np.array(h5['features'])
        labels = np.array(h5['labels'])
        labels[np.where(labels > 0)] = 1
        labels = to_categorical(labels, 2)

print(data.shape)
print(labels.shape)

data_rows, _ = data.shape
print(data_rows)

rand_permed_rows_seed = np.random.permutation(data_rows)

rand_permed_data = data[rand_permed_rows_seed]
rand_permed_labels = labels[rand_permed_rows_seed]

test_data_rows = 500
X_test = rand_permed_data[0:test_data_rows, :]
y_test = rand_permed_labels[0:test_data_rows, :]
X_train = rand_permed_data[test_data_rows: , :]
y_train = rand_permed_labels[test_data_rows: , :]


(2359, 4096)
(2359, 2)
2359


## MLP model

In [136]:
# MLP model (with tanh activation layers)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Create model
model = Sequential()

# Dense(64) is a fully-connected layer with [] hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 4096-dimensional vectors.
model.add(Dense(128, input_dim=4096, init='uniform'))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
'''
model.add(Dense(256, init='uniform'))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
'''
model.add(Dense(2, init='uniform'))
model.add(Activation('softmax'))


# Compile model
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_35 (Dense)                 (None, 128)           524416      dense_input_18[0][0]             
____________________________________________________________________________________________________
activation_35 (Activation)       (None, 128)           0           dense_35[0][0]                   
____________________________________________________________________________________________________
dropout_18 (Dropout)             (None, 128)           0           activation_35[0][0]              
____________________________________________________________________________________________________
dense_36 (Dense)                 (None, 2)             258         dropout_18[0][0]                 
___________________________________________________________________________________________

In [137]:
# Train the MLP model
model.fit(X_train, y_train, batch_size=16, nb_epoch=100, validation_split=0.1)


Train on 1673 samples, validate on 186 samples
Epoch 1/100
1673/1673 [==============================] - 0s - loss: 5.3868 - acc: 0.5935 - val_loss: 6.2914 - val_acc: 0.6075
Epoch 2/100
1673/1673 [==============================] - 0s - loss: 5.4703 - acc: 0.5828 - val_loss: 6.2914 - val_acc: 0.6075

In [138]:
# Test
score = model.evaluate(X_test, y_test, batch_size=16)
score


384/500 [======================>.......] - ETA: 0s

[6.0273699493408204, 0.624]

In [139]:
# MLP model (with tanh sigmoid layers)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Create model
model = Sequential()

# Dense(64) is a fully-connected layer with [] hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 4096-dimensional vectors.
model.add(Dense(128, input_dim=4096, init='uniform'))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
'''
model.add(Dense(256, init='uniform'))
model.add(Activation('sigmoid'))
model.add(Dropout(0.5))
'''
model.add(Dense(2, init='uniform'))
model.add(Activation('softmax'))


# Compile model
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_37 (Dense)                 (None, 128)           524416      dense_input_19[0][0]             
____________________________________________________________________________________________________
activation_37 (Activation)       (None, 128)           0           dense_37[0][0]                   
____________________________________________________________________________________________________
dropout_19 (Dropout)             (None, 128)           0           activation_37[0][0]              
____________________________________________________________________________________________________
dense_38 (Dense)                 (None, 2)             258         dropout_19[0][0]                 
___________________________________________________________________________________________

In [142]:
# Train the MLP model
model.fit(X_train, y_train, batch_size=16, nb_epoch=100, validation_split=0.1)


Train on 1673 samples, validate on 186 samples
Epoch 1/100
1673/1673 [==============================] - 0s - loss: 1.3735 - acc: 0.6724 - val_loss: 0.6242 - val_acc: 0.7688
Epoch 2/100
1673/1673 [==============================] - 0s - loss: 1.3666 - acc: 0.6701 - val_loss: 2.1548 - val_acc: 0.6075

In [143]:
# Test
score = model.evaluate(X_test, y_test, batch_size=16)
score


432/500 [========================>.....] - ETA: 0s

[1.0697292830944061, 0.76000000000000001]

## 1D convolutional model (not yet completed)

In [36]:
# 1D Convolutional model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD

model = Sequential()

# apply a convolution 1d of length 3 to a sequence with 10 timesteps,
# with 64 output filters
model.add(Convolution1D(64, 3, border_mode='same', input_dim=4096, input_length=data_rows))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Convolution1D(32, 3, border_mode='same', input_length=10))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('softmax'))

# Compile model
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_1 (Convolution1D)  (None, 2359, 64)      786496      convolution1d_input_1[0][0]      
____________________________________________________________________________________________________
activation_36 (Activation)       (None, 2359, 64)      0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
dropout_23 (Dropout)             (None, 2359, 64)      0           activation_36[0][0]              
____________________________________________________________________________________________________
convolution1d_2 (Convolution1D)  (None, 2359, 32)      6176        dropout_23[0][0]                 
___________________________________________________________________________________________

In [37]:
# TODO: Fix the problem of input dimension
# Train the 1D convolutional model
model.fit(X_train, y_train, nb_epoch=10, batch_size=16)


Exception: Error when checking model input: expected convolution1d_input_1 to have 3 dimensions, but got array with shape (2359, 4096)